In [1]:
import os
import openai
import pandas as pd
#!pip install sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
df=pd.read_csv('sales_data_sample.csv')

In [4]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [5]:
df.groupby('QTR_ID').sum()['SALES']

/var/folders/s9/hrwvzf4j77g5lgp9_8n3z3s00000gn/T/ipykernel_22043/998531370.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('QTR_ID').sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [6]:
# create tmp db in RAM
# push pandas df into temporay db
# sql query on temp db

In [7]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [8]:
data=df.to_sql(name='Sales', con=temp_db)

2023-10-26 22:00:18,467 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-26 22:00:18,470 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-10-26 22:00:18,470 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-26 22:00:18,472 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-10-26 22:00:18,473 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-26 22:00:18,474 INFO sqlalchemy.engine.Engine ROLLBACK
2023-10-26 22:00:18,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-26 22:00:18,476 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-10-26 22:00:18,477 INFO sqlalche

In [9]:
data

2823

In [10]:
with temp_db.connect() as con:
    result=con.execute(text('SELECT SUM(SALES) from SALES'))

2023-10-26 22:00:19,407 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-26 22:00:19,408 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) from SALES
2023-10-26 22:00:19,410 INFO sqlalchemy.engine.Engine [generated in 0.00267s] ()
2023-10-26 22:00:19,414 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
result.all()

[(10032628.85000001,)]

In [12]:
def create_table_definition(df):
    prompt="""### sqlite SQL tabel, with its properties:
    #
    # Sales({})
    #
    
    """.format(','.join(str(col) for col in df.columns))
    return prompt

In [13]:
print(create_table_definition(df))

### sqlite SQL tabel, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    
    


In [14]:
def prompt_input():
    nlp_text = input("Enter the info you want:")
    return nlp_text

In [71]:
nlp_text = prompt_input()

Enter the info you want: get maximum price each for each month


In [72]:
nlp_text

'get maximum price each for each month'

In [73]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\n SELECT"
    return definition + " " + query_init_string

In [74]:
input_for_openai = combine_prompts(df, nlp_text)
print(input_for_openai)

### sqlite SQL tabel, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    
     ### A query to answer: get maximum price each for each month
 SELECT


In [75]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=input_for_openai,
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=';'
)

In [76]:
print(response['choices'][0]['text'])

 MONTH_ID, MAX(PRICEEACH)
 FROM Sales
 GROUP BY MONTH_ID


In [77]:
def handle_response(response):
    sql_query = response['choices'][0]['text']
    if sql_query.startswith(' '):
        sql_query = 'SELECT'+ sql_query
    return sql_query

In [78]:
test = handle_response(response)

In [79]:
test

'SELECT MONTH_ID, MAX(PRICEEACH)\n FROM Sales\n GROUP BY MONTH_ID'

In [80]:
with temp_db.connect() as con:
    result=con.execute(text(handle_response(response)))

2023-10-26 22:26:02,580 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-26 22:26:02,584 INFO sqlalchemy.engine.Engine SELECT MONTH_ID, MAX(PRICEEACH)
 FROM Sales
 GROUP BY MONTH_ID
2023-10-26 22:26:02,587 INFO sqlalchemy.engine.Engine [generated in 0.00694s] ()
2023-10-26 22:26:02,592 INFO sqlalchemy.engine.Engine ROLLBACK


In [81]:
result.all()

[(1, 100.0),
 (2, 100.0),
 (3, 100.0),
 (4, 100.0),
 (5, 100.0),
 (6, 100.0),
 (7, 100.0),
 (8, 100.0),
 (9, 100.0),
 (10, 100.0),
 (11, 100.0),
 (12, 100.0)]